In [1]:
import torch
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn
import pandas as pd
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
import pandas as pd
train_df = pd.read_csv('train.csv')
submit = pd.read_csv('sample_submission.csv')

In [3]:
# 0 값을 보간하는 함수 정의
def improved_fill_zeros_with_median_and_selective_interpolation_v6(data):
    # 0이 아닌 값들만 추출
    non_zero_data = [value for value in data if value != 0]
    
    # 모든 값이 0인 경우, 원본 데이터 반환
    if len(non_zero_data) == 0: 
        return data
    
    # 0이 아닌 값들의 중앙값 계산
    median_value = np.median(non_zero_data)
    filled_data = data.copy()
    
    # 0이 아닌 값들의 인덱스를 찾음
    non_zero_indices = np.where(data != 0)[0]
    
    # 배열의 시작 부분에 연속된 0 값들을 중앙값과 첫 번째 0이 아닌 값으로 선형 보간
    if non_zero_indices[0] != 0:
        filled_data[:non_zero_indices[0]] = np.linspace(median_value, data[non_zero_indices[0]], non_zero_indices[0]+1)[:-1]
    
    # 배열의 끝 부분에 연속된 0 값들을 마지막 0이 아닌 값과 중앙값으로 선형 보간
    if non_zero_indices[-1] != len(data) - 1:
        filled_data[non_zero_indices[-1]+1:] = np.linspace(data[non_zero_indices[-1]], median_value, len(data)-non_zero_indices[-1])[:-1]
    
    # 0이 아닌 값들 사이의 0 값들을 선형 보간
    for i in range(len(non_zero_indices) - 1):
        start, end = non_zero_indices[i], non_zero_indices[i+1]
        filled_data[start+1:end] = np.linspace(data[start], data[end], end-start+1)[1:-1]
    
    return filled_data

def add_noise_to_interpolated_values_v2(original_data, processed_data, std_dev=0.5):
    """Add Gaussian noise to the interpolated values in the processed data."""
    noise = np.where(original_data == 0, np.random.normal(0, std_dev, len(original_data)), 0)
    noisy_processed_data = processed_data + noise
    return noisy_processed_data

def add_bootstrap_noise_to_interpolated_values(original_data, processed_data):
    """Add bootstrap noise to the interpolated values in the processed data."""
    non_zero_data = [value for value in original_data if value != 0]
    if len(non_zero_data) == 0:
        return processed_data
    noise_values = np.random.choice(non_zero_data, size=len(original_data), replace=True) - np.median(non_zero_data)
    noisy_processed_data = np.where(original_data == 0, processed_data + noise_values, processed_data)
    return noisy_processed_data


In [4]:
train_data = pd.read_csv('train.csv')

for i in range(len(train_data)):
    value = list(train_data.iloc[i][6:])
    interpolated_sales_data = improved_fill_zeros_with_median_and_selective_interpolation_v6(np.array(value))
    train_data.iloc[i, 6:] = interpolated_sales_data
train_data

,ID,제품,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,2,1,1,1,...,3,3,3,3,3,2,2,2,2,2
1,1,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,2,1,1,1,...,1,1,1,1,3,2,2,2,2,2
2,2,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,3,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,3,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,4,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,4,B002-00003-00001,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,5,4,4,4,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,B002-03799-00002,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,7,6,6,6,...,5,5,6,6,6,6,6,6,6,6
15886,15886,B002-03799-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,5,4,4,4,...,3,3,3,3,2,2,4,1,1,3
15887,15887,B002-03799-00004,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,3,3,3,3,...,2,2,2,2,2,2,2,2,2,2
15888,15888,B002-03799-00005,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,3,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [5]:
train_df = train_data.copy()

In [6]:
class LTSF_NLinear(torch.nn.Module):
    def __init__(self, window_size, forecast_size, individual, feature_size):
        super(LTSF_NLinear, self).__init__()
        self.window_size = window_size
        self.forecast_size = forecast_size
        self.individual = individual
        self.channels = feature_size
        if self.individual:
            self.Linear = torch.nn.ModuleList()
            for i in range(self.channels):
                self.Linear.append(torch.nn.Linear(self.window_size, self.forecast_size))
        else:
            self.Linear = torch.nn.Linear(self.window_size, self.forecast_size)

    def forward(self, x):
        seq_last = x[:,-1:,:].detach()
        x = x - seq_last
        if self.individual:
            output = torch.zeros([x.size(0), self.forecast_size, x.size(2)],dtype=x.dtype).to(x.device)
            for i in range(self.channels):
                output[:,:,i] = self.Linear[i](x[:,:,i])
            x = output
        else:
            x = self.Linear(x.permute(0,2,1)).permute(0,2,1)
        x = x + seq_last
        return x

class Data(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]
    

In [7]:
# Function to reshape the data into a time series format for each ID
def reshape_data(df):
    time_series_data = []
    for idx, row in df.iterrows():
        sales_data = row[6:].values.astype(float)
        time_series_data.append(sales_data)
    return np.array(time_series_data)

# Modified time_slide_df function to work with the current data format
def time_slide_df(data, window_size, forecast_size):
    data_list = []
    dap_list = []
    for idx in range(0, len(data) - window_size - forecast_size + 1):
        x = data[idx:idx + window_size].reshape(window_size, 1)
        y = data[idx + window_size:idx + window_size + forecast_size]
        data_list.append(x)
        dap_list.append(y)
    return np.array(data_list, dtype='float32'), np.array(dap_list, dtype='float32')

# Function to create DataLoader for each ID
def create_dataloader(data, window_size, forecast_size, batch_size):
    X, Y = time_slide_df(data, window_size, forecast_size)
    ds = Data(X, Y)
    return DataLoader(ds, batch_size=batch_size, shuffle=True)

# Reshape the data
time_series_data = reshape_data(train_df)

In [8]:
# Function to reshape the data into a time series format for each ID
def reshape_data(df):
    time_series_data = []
    for idx, row in df.iterrows():
        sales_data = row[6:].values.astype(float)
        time_series_data.append(sales_data)
    return np.array(time_series_data)

# Modified time_slide_df function to work with the current data format
def time_slide_df(data, window_size, forecast_size):
    data_list = []
    dap_list = []
    for idx in range(0, len(data) - window_size - forecast_size + 1):
        x = data[idx:idx + window_size].reshape(window_size, 1)
        y = data[idx + window_size:idx + window_size + forecast_size]
        data_list.append(x)
        dap_list.append(y)
    return np.array(data_list, dtype='float32'), np.array(dap_list, dtype='float32')

# Function to create DataLoader for each ID
def create_dataloader(data, window_size, forecast_size, batch_size):
    X, Y = time_slide_df(data, window_size, forecast_size)
    ds = Data(X, Y)
    return DataLoader(ds, batch_size=batch_size, shuffle=True)

# Reshape the data
time_series_data = reshape_data(train_df)

# Define the window size, forecast size, and batch size
window_size = 35   # Considering the last 30 days for prediction
forecast_size = 21 # Predicting the next 21 days
batch_size = 4096
epoch_count = 777
lr = 0.001
min_delta = 0.001
patience = 10

future_predictions_by_id = {}
loss_history = {}

# Iterate through the data by ID
for idx, (id_val, data) in tqdm(enumerate(zip(train_df["ID"], time_series_data)), total=len(train_df["ID"])):
    mean_ = np.mean(data)
    std_ = np.std(data)
    if std_ == 0:
        standardized_data = data
    else:
        standardized_data = (data - mean_) / std_

    individual_loss_history = []
    # Create DataLoader
    train_dl = create_dataloader(standardized_data, window_size, forecast_size, batch_size)
    best_loss = float('inf')
    no_improvement_count = 0

    # Training the model
    DLinear_model = LTSF_NLinear(window_size=window_size, forecast_size=21, individual=False, feature_size=1)
    DLinear_model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(DLinear_model.parameters(), lr=lr)
    for epoch in range(1, epoch_count + 1):
        loss_list = []
        DLinear_model.train()
        for batch_idx, (data, target) in enumerate(train_dl):
            data, target = data.to(device), target.to(device)  # Move the data to the GPU if available
            optimizer.zero_grad()
            output = DLinear_model(data)
            loss = criterion(output, target.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            loss_list.append(loss.item())
        if((epoch % 10) == 0):
            avg_loss = np.mean(loss_list)
            #print(f"Id {idx}, Epoch {epoch}: Loss = {avg_loss}")
            individual_loss_history.append(avg_loss)
            if avg_loss + min_delta < best_loss:
                best_loss = avg_loss
                no_improvement_count = 0
            else:
                no_improvement_count += 1
                if no_improvement_count >= patience:
                    #print(f"Early stopping at epoch {epoch} for ID {id_val}")
                    break


    loss_history[id_val] = individual_loss_history

    # Predicting the future 15 days using the last window of data
    #last_window_data = torch.tensor(standardized_data[-window_size:]).unsqueeze(0).unsqueeze(-1).float()
    last_window_data = torch.tensor(standardized_data[-window_size:]).unsqueeze(0).unsqueeze(-1).float().to(device)
    future_prediction = DLinear_model(last_window_data)

    # Converting the prediction back to the original scale
    #future_prediction = future_prediction.squeeze().detach().numpy() * std_ + mean_
    # Converting the prediction back to the original scale
    if std_ == 0:
        future_prediction = future_prediction.squeeze().detach().cpu().numpy()
    else:
        future_prediction = future_prediction.squeeze().detach().cpu().numpy() * std_ + mean_

    # Store the prediction
    future_predictions_by_id[id_val] = future_prediction

# Future predictions for each ID from 2023-04-05 to 2023-04-25
future_predictions_by_id

  0%|          | 19/15890 [02:01<28:17:27,  6.42s/it]


KeyboardInterrupt: 

In [9]:
# Filling the submission DataFrame with the predicted values for 21 days (converted to float64)
for id_val, predictions in future_predictions_by_id.items():
    rounded_predictions = np.round(predictions).astype(np.float64)
    submit.loc[submit['ID'] == id_val, '2023-04-05':'2023-04-25'] = rounded_predictions.astype(np.float64)

# Displaying the first few rows of the filled submission file
submit.head(21)

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,2,3,3,3,3,4,4,4,4,...,5,5,5,5,5,5,5,6,6,6
1,1,2,2,2,2,2,2,2,2,1,...,1,1,1,1,1,2,2,1,1,1
2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
5,5,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
6,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
7,7,44,44,43,43,43,43,42,42,42,...,43,43,43,43,42,42,42,42,41,41
8,8,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9,9,6,7,7,7,7,7,7,7,8,...,7,8,8,8,8,8,8,8,8,8


In [10]:
submit.iloc[:, 1:] = submit.iloc[:, 1:].applymap(lambda x: 0 if x < 0 else x)
submit

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
15886,15886,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,3.0,4.0,4.0,3.0,4.0,4.0
15887,15887,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
15888,15888,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [11]:
submit = submit.fillna(0)

In [12]:
submit.to_csv('NLinear_35_.csv',index=False)
submit

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
15886,15886,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,3.0,4.0,4.0,3.0,4.0,4.0
15887,15887,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
15888,15888,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
